In [1]:
import gc
import tensorflow as tf
import tensorflow_datasets
import numpy as np
import tensorflow.keras as keras
import orjson
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Dense, Input
from utils import downconvert_tf_dataset
import wandb
from wandb.keras import WandbCallback

from transformers import (TFBertModel, TFDistilBertModel,
                          BertTokenizer, DistilBertTokenizer,
                          glue_convert_examples_to_features)

# FP16 settings
fp16 = False
if fp16:
    tf.config.optimizer.set_experimental_options({"auto_mixed_precision": True})

In [2]:
def split_inputs_and_outputs(data_map):
    return [np.asarray(data_map['input_id']),
           np.asarray(data_map['attention_mask']),
           np.asarray(data_map['token_type_id'])], np.asarray(data_map['label'])

def split_inputs_and_outputs_distil(data_map):
    return [np.asarray(data_map['input_id']),
           np.asarray(data_map['attention_mask'])], np.asarray(data_map['label'])

def load_data(train_filename, val_filename):
    training_data = orjson.loads(open(train_filename, "rb").read())
    #train_x, train_y = split_inputs_and_outputs(training_data)
    train_x, train_y = split_inputs_and_outputs_distil(training_data)
    val_data = orjson.loads(open(val_filename, "rb").read())
    val_x, val_y = split_inputs_and_outputs(val_data)
    return train_x, train_y, val_x, val_y

# Load data.
train_dataset_path = "C:/Users/jbetk/Documents/data/ml/sentiment_analysis/outputs/processed.json"
val_dataset_path = "C:/Users/jbetk/Documents/data/ml/sentiment_analysis/outputs/validation.json"
train_x, train_y, val_x, val_y = load_data(train_dataset_path, val_dataset_path)

In [3]:
def create_new_classification_head(base_model_cls_head, dense_config=[256,2], final_activation="softmax"):   
    # Create the head.
    tensor = base_model_cls_head
    for layer_units in dense_config[0:-1]:
        tensor = Dense(units=layer_units, activation="relu")(tensor)
    tensor = Dense(units=dense_config[-1], activation=final_activation, name="final_linear")(tensor)
    
    return tensor

def fine_tune_task(optimizer):
    
    # Re-load base model weights.
    bert_base_model = TFDistilBertModel.from_pretrained("distilbert-base-uncased")
    #bert_base_model = TFBertModel.from_pretrained("bert-base-cased")

    inputs = [Input(shape=(128,), dtype='int32', name='input_ids'),
              Input(shape=(128,), dtype='int32', name='attention_mask'), 
              #Input(shape=(128,), dtype='int32', name='token_type_ids'),
             ]

    # Fetch the CLS head of the BERT model; index 1.
    #cls_head = bert_base_model(inputs)[1]
    # Classification head for DistilBERT is the flattened hidden state.
    cls_head = keras.layers.Flatten()(bert_base_model(inputs)[0])
    
    tensor = create_new_classification_head(cls_head, dense_config=[1], final_activation="linear")
    model = keras.Model(inputs=inputs, outputs=tensor)
    print(model.summary())

    # Configure loss function and metrics.
    if fp16:
        tf.train.experimental.enable_mixed_precision_graph_rewrite(optimizer)
    #loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    loss = tf.keras.losses.MeanSquaredError()
    metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

    model.compile(optimizer=optimizer, loss=loss, metrics=[metric])
    
    return model

In [4]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
#tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5, epsilon=1e-08)

batch_sz = 32
epochs = 2

# Todo - configure optimizer in mixed precision mode.
model = fine_tune_task(optimizer)

# Train model.
wandb.init(project="nonint-transformers",\
           name="amazon_reviews_sentiment_first_pass",\
           config={"dataset": "amazon_yelp_dataset", "learning_rate": 1e-5, "epsilon": 1e-08, "batch_sz": batch_sz})

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 128)]        0                                            
__________________________________________________________________________________________________
attention_mask (InputLayer)     [(None, 128)]        0                                            
__________________________________________________________________________________________________
tf_distil_bert_model (TFDistilB ((None, 128, 768),)  66362880    input_ids[0][0]                  
                                                                 attention_mask[0][0]             
__________________________________________________________________________________________________
flatten (Flatten)               (None, 98304)        0           tf_distil_bert_model[0][0]   

W&B Run: https://app.wandb.ai/neonbjb/nonint-transformers/runs/oje9lms7

In [ ]:
history = model.fit(train_x, train_y, batch_size=batch_sz, epochs=epochs, \
                                      validation_data=(val_x, val_y),\
                                      callbacks=[WandbCallback(log_batch_frequency=int(1024/batch_sz))])

Train on 2625860 samples, validate on 4000 samples
Epoch 1/2
2625856/2625860 [============================>.] - ETA: 0s - loss: 1.1052 - accuracy: 0.0000e+00

wandb: ERROR Can't save model, h5py returned error: 


2625860/2625860 [==============================] - 15060s 6ms/sample - loss: 1.1052 - accuracy: 0.0000e+00 - val_loss: 0.9724 - val_accuracy: 0.0000e+00
Epoch 2/2
 839104/2625860 [========>.....................] - ETA: 2:49:09 - loss: 1.1185 - accuracy: 0.0000e+00- ETA: 2:49:09 - loss: 1.1185 - accuracy: 0.0000e

In [6]:
phrases = [
    "This product ruined my day. Not figuratively, I stuck my fingers together and couldn't get them apart! I would never recommend this to anyone! Luckily I could return it..",
    # 5 star:
    "This product has been the standard for woodworking pros for decades. Before the 800lb gorilla swaggered into the room, taking the market by storm and strong-arming his way to happily humming registers all over the country, Titebond was THE glue to beat.",
    # 4 star:
    "good glue, but had hardened glue stuck in the spout and is clogged have to unscrew and use a popsicle stick to use, i will try and use isopropyl to dissolve it so i can use the spout, kinda annoying",
    # 3 star:
    "Strong as can be, but stain will discolor. Make sure you wipe it off completely before it dries.",
    # 2 star:
    "Did not use this glue immediately. About 4 weeks later, I could not get any glue to squeeze out thru the dispenser tip. I removed the cap and discovered lumps in the glue.",
]

def pad_zero(inputs, seq_len):
    for k in inputs: 
        output = np.zeros(seq_len, dtype='int32')
        output[:len(inputs[k])] = np.asarray(inputs[k])
        inputs[k] = output
    return inputs

phrases_encoded = [[], [], []]
for phrase in phrases:
    enc = pad_zero(tokenizer.encode_plus(phrase, add_special_tokens=True, max_length=128), 128)
    for (i,k) in enumerate(enc.keys()):
        phrases_encoded[i].append(enc[k])

inputs = []
for e in phrases_encoded:
    inputs.append(np.asarray(e))
    
results = model.predict(inputs)

for p,r in zip(phrases, results):
    print(p, r)

This product ruined my day. Not figuratively, I stuck my fingers together and couldn't get them apart! I would never recommend this to anyone! Luckily I could return it.. [1.6857811]
This product has been the standard for woodworking pros for decades. Before the 800lb gorilla swaggered into the room, taking the market by storm and strong-arming his way to happily humming registers all over the country, Titebond was THE glue to beat. [2.3921506]
good glue, but had hardened glue stuck in the spout and is clogged have to unscrew and use a popsicle stick to use, i will try and use isopropyl to dissolve it so i can use the spout, kinda annoying [1.1922858]
Strong as can be, but stain will discolor. Make sure you wipe it off completely before it dries. [1.5189675]
Did not use this glue immediately. About 4 weeks later, I could not get any glue to squeeze out thru the dispenser tip. I removed the cap and discovered lumps in the glue. [1.8800578]


In [7]:
tf.saved_model.save(model,"saved/")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: saved/assets
